In [1]:
"""
This notebook evaluates the trained model on the *.ply files stored in the config.root folder.
"""

# Script Parameters: path to weights and data (folder containing .ply files)
# Point clouds are downsampled with a spatial sampling of 5mm.
# Scenes are limited to a maximum distance of 5m from the acquisition device.


# config.job_name contains the name of the folder containing the weights in the save directory.
PATH_TO_WEIGHTS = "/path/to/model/weights/"

# Path to data (.ply files, each containing a station, downsampled at 5mm).
PATH_TO_DATA = "/path/to/data/"

import glob
import os
import sys
import random
import numpy as np

BASE_DIR = os.path.dirname(os.path.abspath("./"))
ROOT_DIR = os.path.dirname(BASE_DIR)
sys.path.append(ROOT_DIR)

import torch

from models import build_scene_segmentation




from evaluate_utils import create_cfg

# Local GPU ID to be used.
device_id = 0
device = "cuda:{}".format(device_id)

diameter_percent = 40

# path2basecfgs: path to .cfg (.yml) configuration files located at the root of the repository.
config = create_cfg(diameter_percent=diameter_percent,path2plyclouds=PATH_TO_DATA,path2basecfgs="./cfgs/")

# Seeds definition
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
random.seed(0)
np.random.seed(0)

# Setting the device ID
torch.cuda.set_device(device_id)

# Path to be set in the environment variables so that build_scene_segmentation can access the positions of the KPConv kernel points.
os.environ["JOB_LOAD_DIR"] = PATH_TO_WEIGHTS

Num. points: 15000
RADII: in_radius=2.00, smallest radius=0.10825
SAMPLEDL = 0.06


In [2]:
# Model and loss creation.
model, criterion = build_scene_segmentation(config)

# Loading the checkpoint.
checkpoint = torch.load(os.environ["JOB_LOAD_DIR"]+"BEST.pth", map_location='cpu')

# Necessary operation because the training was done via the torch multi-GPU environment --> the model was encapsulated in a "module" object that needs to be manually removed from the weight dictionary keys.
orig_dict = checkpoint['model']
model_weights = {}
for k in orig_dict.keys():
    model_weights[k.replace("module.","")] = orig_dict[k]
model.load_state_dict(model_weights)

# Using the GPU
model = model.to(device)

/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy
============================>/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy loaded <=======================
/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy
============================>/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy loaded <=======================
/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy
============================>/gpfsgaia/projets/maquette3d/POIDS_MODELES/finetuned_EDFM_grid_040_int_ktz_std_3.20/kernels/dispositions/sk_pt_0.064952_015_center.npy loaded <=======================
/gpfsga

In [3]:
from RUNME_utils import process_cloud,run_inference_on_cloud
from data_utils import write_ply
import os

# Recovery of cloud names in the path
cloud_paths = glob.glob(config.data_root+"/*.ply")

batch_size = 8

num_clouds = len(cloud_paths)
for cloud_id,cloud_path in enumerate(cloud_paths):
    # Definition of some utility variables
    cloud_name = os.path.basename(cloud_path)
    cloud_dir = cloud_path.replace(cloud_name,"")
    out_file = "{}_inferred.ply".format(cloud_name.replace(".ply",""))
    
    if not os.path.exists(out_file):
        print("[{:03d}/{:03d}] Starting cloud {} in {}.".format(cloud_id+1,num_clouds,cloud_name,cloud_dir,))
        
        # This function preprocesses the point cloud
        # hd_points: numpy array of size (N, 3) corresponding to points at "HD" downsampling (i.e. one point every 5mm).
        # NN: numpy array of size (N,) storing the indices of low-resolution cloud points that are closest to the high-resolution cloud points. The low-resolution point cloud (with M points) is the one on which the inference is performed. To retrieve the prediction at the highest resolution (with N points), these indices need to be used. This operation is automatically performed in the run_inference_on_cloud function.
        # cur_features: numpy array of size (M, F) corresponding to F features used, downsampled to the low-resolution cloud.
        # cur_tree: KD tree of the low-resolution cloud (M, 3). The cur_tree.data field gives access to the points of the low-resolution cloud.
        # patch_center_indices: indices of the patch centers where the inference is executed. There is one patch center every 0.25 times the patch radius to cover the cloud effectively.
        hd_points,NN,cur_features,cur_tree,patch_center_indices = process_cloud(cloud_path,config)

        print("\tPreprocessing done.")

        # Recovery of the probability distribution probas_01 on the high-resolution cloud.
        # probas_01: numpy array of size (N, 1).
        probas_01 = run_inference_on_cloud(config,model,batch_size,patch_center_indices,hd_points,cur_tree,cur_features,NN)

        # Saving the result to a .ply file.
        write_ply(out_file,[hd_points,probas_01],["vertex","probas_01"])

        print("Done.")

[002/040] Starting cloud CRU3BR08_R849_8025_SPATIAL_SUBSAMPLED_5mm.ply in /gpfsgaia/projets/maquette3d/_TEST_DATA/.
	Preprocessing done.
	Inference on batch 0001/0109
	Inference on batch 0002/0109
	Inference on batch 0003/0109
	Inference on batch 0004/0109
	Inference on batch 0005/0109
	Inference on batch 0006/0109
	Inference on batch 0007/0109
	Inference on batch 0008/0109
	Inference on batch 0009/0109
	Inference on batch 0010/0109
	Inference on batch 0011/0109
	Inference on batch 0012/0109
	Inference on batch 0013/0109
	Inference on batch 0014/0109
	Inference on batch 0015/0109
	Inference on batch 0016/0109
	Inference on batch 0017/0109
	Inference on batch 0018/0109
	Inference on batch 0019/0109
	Inference on batch 0020/0109
	Inference on batch 0021/0109
	Inference on batch 0022/0109
	Inference on batch 0023/0109
	Inference on batch 0024/0109
	Inference on batch 0025/0109
	Inference on batch 0026/0109
	Inference on batch 0027/0109
	Inference on batch 0028/0109
	Inference on batch 002